#Imports


In [ ]:
#!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-1.11.0+cu113.html
#!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-1.11.0+cpu.html
#!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-1.11.0+cu113.html
#!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-1.11.0+cpu.html
#!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

In [ ]:
import torch_geometric
from torch_geometric.data import Data
import torch
from torch.utils.data import DataLoader
from torch.optim import optimizer

In [ ]:
#!pip install chompack

import chompack as cp
from cvxopt import spmatrix, amd, matrix, sparse


import numpy as np
import scipy.io


import pandas as pd

import os
import numpy as np
from numpy import savez_compressed
from numpy import load
import matplotlib.pyplot as plt
from scipy.sparse import diags


from PIL import Image
from IPython.display import display

#Read in matrices

In [ ]:
# This function returns the fill-in count for a matrix
def getFillIn(mat):
  # Convert the numpy mat to a cvxopt sparse matrix
  spmat = sparse(matrix(mat.astype(np.double)))

  # By default does not perform any permutations
  symb = cp.symbolic(spmat)
  fillIn = symb.fill[0]
  fillIn = symb.nnz
  #print(mat)
  #print(symb)
  #print(symb.fill[0], symb.fill[1], symb.nnz)

  return int(fillIn)

In [ ]:
fpprec = 'float32'

filename = '2_512_mat_nnz.mat'

print('Max NNZ:', 128*128)

loaded = scipy.io.loadmat(filename)
#[data][0-flatten][order][0-flatten]  order: 0-rcm, 1-amd, 2-nd, 3-nat
x_train_rcm = loaded['x_train'][0][0][0]
y_train_rcm = loaded['y_train'][0][0][0]
x_train_amd = loaded['x_train'][0][1][0]
y_train_amd = loaded['y_train'][0][1][0]
x_train_nd  = loaded['x_train'][0][2][0]
y_train_nd  = loaded['y_train'][0][2][0]
x_train     = loaded['x_train'][0][3][0]
y_train     = loaded['y_train'][0][3][0]

x_test_rcm = loaded['x_test'][0][0][0]
y_test_rcm = loaded['y_test'][0][0][0]
x_test_amd = loaded['x_test'][0][1][0]
y_test_amd = loaded['y_test'][0][1][0]
x_test_nd  = loaded['x_test'][0][2][0]
y_test_nd  = loaded['y_test'][0][2][0]
x_test     = loaded['x_test'][0][3][0]
y_test     = loaded['y_test'][0][3][0]



for i in range(len(x_train_rcm)):
    x_train_rcm[i] = x_train_rcm[i].astype('float32')
for i in range(len(x_train_amd)):
    x_train_amd[i] = x_train_amd[i].astype('float32')
for i in range(len(x_train_nd)):
    x_train_nd[i] = x_train_nd[i].astype('float32')
for i in range(len(x_train)):
    x_train[i] = x_train[i].astype('float32')
for i in range(len(x_test_rcm)):
    x_test_rcm[i] = x_test_rcm[i].astype('float32')
for i in range(len(x_test_amd)):
    x_test_amd[i] = x_test_amd[i].astype('float32')
for i in range(len(x_test_nd)):
    x_test_nd[i] = x_test_nd[i].astype('float32')
for i in range(len(x_test)):
    x_test[i] = x_test[i].astype('float32')
    
print(x_train_rcm)
print(x_train_rcm[0].dtype)
print(y_train_rcm[0].dtype)
    



In [ ]:
#Test order
testi = 2
print('NNZ. NAT: ', y_train[testi], ' RCM ', y_train_rcm[testi], ' AMD ', y_train_amd[testi], ' ND ', y_train_nd[testi])





#Remove duplicates

In [ ]:
def remove_dup(x,y):
    dup_counter = 0;
    skip_list = [0]*len(x);
    print('skip list len', len(x), len(skip_list))
    new_x = [];
    new_y = [];
    for i in range(len(x)):
        if(skip_list[i] == 0):
            for j in range(i+1, len(x)):
                if np.array_equal(x[i], x[j]):
                    skip_list[j] = 1;
                    dup_counter += 1
            new_x.append(x[i]);
            new_y.append(y[i]);
            
    print('Duplicates removed: ', dup_counter)
    new_size = len(x) - dup_counter;
    print('new size: ', new_size, len(new_x))
    
    return new_x, new_y
    #return np.array(new_x), np.array(new_y)
                

In [ ]:
x_train_rcm, y_train_rcm = remove_dup(x_train_rcm, y_train_rcm);
x_train_amd, y_train_amd = remove_dup(x_train_amd, y_train_amd);
x_train_nd, y_train_nd   = remove_dup(x_train_nd, y_train_nd);
x_train, y_train         = remove_dup(x_train, y_train);
x_test_rcm, y_test_rcm   = remove_dup(x_test_rcm, y_test_rcm);
x_test_amd, y_test_amd   = remove_dup(x_test_amd, y_test_amd);
x_test_nd,  y_test_nd    = remove_dup(x_test_nd, y_test_nd);
x_test, y_test           = remove_dup(x_test, y_test);

In [ ]:
def rebalance(Atrain, ytrain, Atest, ytest):
    ratio = .80
    Atemp = Atrain + Atest;
    ytemp = ytrain+ ytest;
    Atrain_out = [];
    ytrain_out = [];
    Atest_out = [];
    ytest_out = [];
    print('length: ', len(Atemp), len(ytemp));
    randp = np.random.permutation(len(ytemp));
    
    ratio = len(ytemp)*ratio
    for i in range(len(ytemp)):
        if(i <= ratio):
            Atrain_out.append(Atemp[i]);
            ytrain_out.append(ytemp[i]);
        else:
            Atest_out.append(Atemp[i]);
            ytest_out.append(ytemp[i]);
            
    print('end length: ', len(ytrain_out), len(ytest_out) );
    
    
    return np.array(Atrain_out), np.array(ytrain_out), np.array(Atest_out), np.array(ytest_out);
    #return Atrain_out, ytrain_out, Atest_out, ytest_out;
            

In [ ]:
x_train_rcm, y_train_rcm, x_test_rcm, y_test_rcm = rebalance(x_train_rcm, y_train_rcm, x_test_rcm, y_test_rcm);
x_train_amd, y_train_amd, x_test_amd, y_test_amd = rebalance(x_train_amd, y_train_amd, x_test_amd, y_test_amd);
x_train_nd, y_train_nd, x_test_nd, y_test_nd     = rebalance(x_train_nd, y_train_nd, x_test_nd, y_test_nd);
x_train, y_train, x_test, y_test                 = rebalance(x_train, y_train, x_test, y_test);


#Visualize the fill-in distribution of the dataset

In [ ]:
# Visualize the distribution of training and test set fill-in counts
fig, ax = plt.subplots(1, figsize=(15,7))
numBins = 100
normalize = True

# Distributions should appear about the same
# Training set is in BLUE, testing set is in RED
ax.hist(y_train_rcm, bins=numBins, density=normalize, 
        fc=(0, 0, 1, 0.5), label='training set')

ax.hist(y_test_rcm,  bins=numBins, density=normalize, 
        fc=(1, 0, 0, 0.5), label='testing set')

ax.legend()
ax.set_xlabel('Fill-in Count')
ax.set_ylabel('Normalized Histogram Counts')
ax.set_title('Training and Test Set Fill-In Histograms -- RCM')

plt.show()

# Visualize the distribution of training and test set fill-in counts
fig, ax = plt.subplots(1, figsize=(15,7))
numBins = 100
normalize = True

# Distributions should appear about the same
# Training set is in BLUE, testing set is in RED
ax.hist(y_train_amd, bins=numBins, density=normalize, 
        fc=(0, 0, 1, 0.5), label='training set')

ax.hist(y_test_amd,  bins=numBins, density=normalize, 
        fc=(1, 0, 0, 0.5), label='testing set')

ax.legend()
ax.set_xlabel('Fill-in Count')
ax.set_ylabel('Normalized Histogram Counts')
ax.set_title('Training and Test Set Fill-In Histograms -- AMD')

plt.show()

# Visualize the distribution of training and test set fill-in counts
fig, ax = plt.subplots(1, figsize=(15,7))
numBins = 100
normalize = True

# Distributions should appear about the same
# Training set is in BLUE, testing set is in RED
ax.hist(y_train_nd, bins=numBins, density=normalize, 
        fc=(0, 0, 1, 0.5), label='training set')

ax.hist(y_test_nd,  bins=numBins, density=normalize, 
        fc=(1, 0, 0, 0.5), label='testing set')

ax.legend()
ax.set_xlabel('Fill-in Count')
ax.set_ylabel('Normalized Histogram Counts')
ax.set_title('Training and Test Set Fill-In Histograms -- ND')

plt.show()

# Visualize the distribution of training and test set fill-in counts
fig, ax = plt.subplots(1, figsize=(15,7))
numBins = 100
normalize = True

# Distributions should appear about the same
# Training set is in BLUE, testing set is in RED
ax.hist(y_train, bins=numBins, density=normalize, 
        fc=(0, 0, 1, 0.5), label='training set')

ax.hist(y_test,  bins=numBins, density=normalize, 
        fc=(1, 0, 0, 0.5), label='testing set')

ax.legend()
ax.set_xlabel('Fill-in Count')
ax.set_ylabel('Normalized Histogram Counts')
ax.set_title('Training and Test Set Fill-In Histograms -- Natural')

plt.show()


#Build Graph

In [ ]:
device = torch.device('cuda')

#Build Graph
#0->1, 0->2, 0->3, ...., 0->n (i.e., 0->127)
#1->2, 1->3, ....., 1->n
#etc
N = 512;

edge_index = torch.tensor([[N-2,N-1]], dtype=torch.long)
for out_edge in range(N-2):
  for in_edge in range(out_edge+1, N):
    #print('out', out_edge, 'in', in_edge)
    temp = torch.tensor([[out_edge, in_edge]], dtype=torch.long)
    edge_index = torch.cat((edge_index,temp))
    #print(edge_index)

edge_index = edge_index.t().contiguous() #need transpose
edge_index = edge_index.to(device)


print(x_train_rcm[0])

data_rcm = Data(x=x_train_rcm[1], edge_index=edge_index, num_nodes=512, num_node_features=512)
data_amd = Data(x=x_train_amd[1], edge_index=edge_index, num_nodes=512, num_node_features=512)
data_nd  = Data(x=x_train_nd[1], edge_index=edge_index, num_nodes=512, num_node_features=512)
data_nat  = Data(x=x_train[1], edge_index=edge_index, num_nodes=512, num_node_features=512)

data_rcm = data_rcm.to(device)
data_amd = data_amd.to(device)
data_nd  = data_nd.to(device)
data_nat = data_nat.to(device)

print(data_rcm)
print(data_amd)
print(data_nd)
print(data_nat)
print('number of nodes', data_rcm.num_nodes)
print('number of edges' , data_rcm.num_edges)
print('number of features', data_rcm.num_node_features)
#print(data.num_classes)
print(data_rcm.has_isolated_nodes())
print(data_rcm.is_directed())
print(data_rcm.has_self_loops())

#Make NN Model

In [ ]:
h1_size = 32
h2_size = 16

class GModel(torch.nn.Module):
  def __init__(self, data):
    super().__init__()
    self.layer1 = torch.nn.GRU(512,512,num_layers=1, batch_first=True, 
                              bias=False, dropout=.1, bidirectional=False)
    self.layer3 = torch.nn.Linear(512,1, bias=False)
    self.layer4 = torch.nn.Linear(512,1, bias=False)
    
    torch.nn.init.ones_(self.layer3.weight)
    torch.nn.init.ones_(self.layer4.weight)
    
  def forward(self, x, data):
    #edge_index =  data.edge_index
    #print(x)
    #print(edge_index)
    #x = torch.reshape(x, (1,512*512))
    x, hidden = self.layer1(x)
    #print('after first', x.shape)
    x = self.layer3(x)
    #print('after two', x.shape)
    x = self.layer4(x.t().contiguous())
    
    #print(x.shape)
    #x = self.layer3(x)
    #print(x)
    return x


#Train Visual Helper

In [ ]:
from torch._C import set_num_interop_threads
def check_acc(loader, model, data):
  diff_fun = torch.nn.L1Loss()
  min = 9999999;
  max = 0;
  max_fill = 0;
  suml1 = 0
  avgl1 = 0
  ntotal = 0
  model.eval();
  with torch.no_grad():
    for index,  (x,y) in enumerate(loader):
      #print('index', index)
      ntotal += 1
      x = torch.squeeze(x)
      y = torch.squeeze(y)
      scores = model(x, data);
      scores = scores.squeeze();
      #print(scores.size())
      if(index == 0 ):
        #print('first')
        y_val = torch.tensor([scores]);
        y_val_test = torch.tensor([y]);
      else:
        #print('second')
        #print('scores', scores.shape, 'yval', y_val.shape)
        y_val = torch.cat((y_val,torch.tensor([scores])))
        y_val_test = torch.cat((y_val_test,torch.tensor([y])))
        #print(y_val, y_val_test)
      #print(y_val.shape)
      loss = diff_fun(scores,y)
      loss_hand = torch.abs(torch.subtract(scores,y))
      #print(loss_hand.item()
      suml1 += loss_hand.item()
      if loss < min:
        min = loss;
      if loss > max:
        max = loss
      #print(torch.max(y))
      if torch.max(y).item() > max_fill:
        max_fill = torch.max(y).item()
      #print('loss', loss)

  print('max:', max, 'min:', min, 'suml1', suml1, 'avg', suml1/ntotal)
  model.train()


  #print(y_val)
  #print(y_val_test)

  fig = plt.figure(figsize=(6,6));
  plt.plot([0,max_fill+100], [0,max_fill+100], '--')
  plt.scatter(y_val_test,y_val, c='blue');
  plt.xlim(0,max_fill+100)
  plt.ylim(0, max_fill+100)
  plt.xlabel('True Fill-in')
  plt.ylabel('Predicted Fill-in')
  plt.show()
  return




#Build DataLoader

In [ ]:
class MyDataset(torch.utils.data.Dataset):
  def __init__(self, x, y):
    super(MyDataset,self).__init__()
    self.x = x;
    self.y = y;

  def __len__(self):
    return len(self.y)


  def __getitem__(self,index):
    return self.x[index], self.y[index]

In [ ]:
#natural 
x_train = torch.tensor(x_train).to(device)
y_train = torch.tensor(y_train).to(device)
x_test  = torch.tensor(x_test).to(device)
y_test  = torch.tensor(y_test).to(device)
train_dataset_nat = MyDataset(x_train, y_train)
train_loader_nat = DataLoader(dataset=train_dataset_nat, batch_size=1, shuffle=True)
test_dataset_nat = MyDataset(x_test, y_test)
test_loader_nat = DataLoader(dataset=test_dataset_nat, batch_size=1, shuffle=False)
#rcm
y_train_rcm = torch.tensor(y_train_rcm).to(device)
y_test_rcm  = torch.tensor(y_test_rcm).to(device)
train_dataset_rcm = MyDataset(x_train, y_train_rcm)
train_loader_rcm = DataLoader(dataset=train_dataset_rcm, batch_size=1, shuffle=True)
test_dataset_rcm = MyDataset(x_test, y_test_rcm)
test_loader_rcm = DataLoader(dataset=test_dataset_rcm, batch_size=1, shuffle=False)
#amd
y_train_amd = torch.tensor(y_train_amd).to(device)
y_test_amd  = torch.tensor(y_test_amd).to(device)
train_dataset_amd = MyDataset(x_train, y_train_amd)
train_loader_amd = DataLoader(dataset=train_dataset_amd, batch_size=1, shuffle=True)
test_dataset_amd = MyDataset(x_test, y_test_amd)
test_loader_amd = DataLoader(dataset=test_dataset_amd, batch_size=1, shuffle=False)
#nd
y_train_nd = torch.tensor(y_train_nd).to(device)
y_test_nd  = torch.tensor(y_test_nd).to(device)
train_dataset_nd = MyDataset(x_train, y_train_nd)
train_loader_nd = DataLoader(dataset=train_dataset_nd, batch_size=1, shuffle=True)
test_dataset_nd = MyDataset(x_test, y_test_nd)
test_loader_nd = DataLoader(dataset=test_dataset_nd, batch_size=1, shuffle=False)

#Train Nat Model

In [ ]:
%%time
nepoch = 200;

net_nat = GModel(data_nat)
net_nat = net_nat.to(device)
print(net_nat)
#xtemp = torch.from_numpy(x_train[0]);
#print(xtemp)
#output = net(xtemp, data)
#print('output test', output)


mycrit = torch.nn.SmoothL1Loss();
#mycrit = torch.nn.MSELoss();
#myoptimizer_nat = torch.optim.RMSprop(net_nat.parameters());
myoptimizer_nat = torch.optim.Adagrad(net_nat.parameters());
#myoptimizer = torch.optim.Adam(net.parameters(), lr=0.01, weight_decay=5e-4)

net_nat.train()

for epoch in range(nepoch):
  loss = 0;
  for batch_idx, (xtemp, ytemp) in enumerate(train_loader_nat):
   
    #forward step
    #print('one', xtemp)
    xtemp = torch.squeeze(xtemp)
    #print('two', xtemp)
    scores = net_nat(xtemp, data_nat)
    #remove extra tensor layers
    scores = torch.squeeze(scores)
    ytemp = torch.squeeze(ytemp)
    loss = mycrit(scores, ytemp) 

    #back step
    myoptimizer_nat.zero_grad();
    loss.backward();
    myoptimizer_nat.step();

  check_acc(test_loader_nat,net_nat, data_nat)
  print('epoch:', epoch, 'loss:', loss)

#Test Nat Model

In [ ]:
%%time
#save
#torch.save(net_nat.state_dict(), 'net_nat_gru.model')
net_nat = GModel(data_nat)
net_nat = net_nat.to(device)
net_nat.load_state_dict(torch.load('net_nat_gru.model'))
#check

check_acc(train_loader_nat, net_nat, data_nat)
check_acc(test_loader_nat, net_nat, data_nat)

#Train RCM Model

In [ ]:
%%time
#nepoch = 100;

net_rcm = GModel(data_rcm)
net_rcm = net_rcm.to(device)
print(net_rcm)
#xtemp = torch.from_numpy(x_train[0]);
#print(xtemp)
#output = net(xtemp, data)
#print('output test', output)


mycrit = torch.nn.SmoothL1Loss();
#mycrit = torch.nn.MSELoss();
#myoptimizer_rcm = torch.optim.RMSprop(net_rcm.parameters());
myoptimizer_rcm = torch.optim.Adagrad(net_rcm.parameters());
#myoptimizer = torch.optim.Adam(net.parameters(), lr=0.01, weight_decay=5e-4)

net_rcm.train()

for epoch in range(nepoch):
  loss = 0;
  for batch_idx, (xtemp, ytemp) in enumerate(train_loader_rcm):
   
    #forward step
    #print('one', xtemp)
    xtemp = torch.squeeze(xtemp)
    #print('two', xtemp)
    scores = net_rcm(xtemp, data_rcm)
    #remove extra tensor layers
    scores = torch.squeeze(scores)
    ytemp = torch.squeeze(ytemp)
    loss = mycrit(scores, ytemp) 

    #back step
    myoptimizer_rcm.zero_grad();
    loss.backward();
    myoptimizer_rcm.step();

  check_acc(test_loader_rcm,net_rcm, data_rcm)
  print('epoch:', epoch, 'loss:', loss)

#Test RCM Model

In [ ]:
%%time
#save
#torch.save(net_rcm.state_dict(), 'net_rcm_gru.model')
net_rcm = GModel(data_rcm)
net_rcm = net_rcm.to(device)
net_rcm.load_state_dict(torch.load('net_rcm_gru.model'))
#check

check_acc(train_loader_rcm, net_rcm, data_rcm)
check_acc(test_loader_rcm, net_rcm, data_rcm)

#check_acc(train_loader_nat, net_nat, data_nat)
#check_acc(test_loader_nat, net_nat, data_nat)

#Train AMD Model

In [ ]:
%%time
nepoch = 200;

net_amd = GModel(data_amd)
net_amd = net_amd.to(device)
print(net_amd)
#xtemp = torch.from_numpy(x_train[0]);
#print(xtemp)
#output = net(xtemp, data)
#print('output test', output)


#mycrit = torch.nn.MSELoss();
mycrit = torch.nn.SmoothL1Loss();
#myoptimizer_amd = torch.optim.RMSprop(net_amd.parameters());
myoptimizer_amd = torch.optim.Adagrad(net_amd.parameters());
#myoptimizer = torch.optim.Adam(net.parameters(), lr=0.01, weight_decay=5e-4)

net_amd.train()

for epoch in range(nepoch):
  loss = 0;
  for batch_idx, (xtemp, ytemp) in enumerate(train_loader_amd):
   
    #forward step
    #print('one', xtemp)
    xtemp = torch.squeeze(xtemp)
    #print('two', xtemp)
    scores = net_amd(xtemp, data_amd)
    #remove extra tensor layers
    scores = torch.squeeze(scores)
    ytemp = torch.squeeze(ytemp)
    loss = mycrit(scores, ytemp) 

    #back step
    myoptimizer_amd.zero_grad();
    loss.backward();
    myoptimizer_amd.step();

  check_acc(test_loader_amd,net_amd, data_amd)
  print('epoch:', epoch, 'loss:', loss)

#Test AMD Model

In [ ]:
%%time
#save
torch.save(net_amd.state_dict(), 'net_amd_gru.model')
#net_amd = GModel(data_amd)
#net_amd.load_state_dict(torch.load('net_amd_smooth_ada_200_512.model'))
#check
check_acc(train_loader_amd, net_amd, data_amd)
check_acc(test_loader_amd, net_amd, data_amd)

#Train ND Model

In [ ]:
#nepoch = 100;

net_nd = GModel(data_nd)
net_nd = net_nd.to(device)
print(net_nd)
#xtemp = torch.from_numpy(x_train[0]);
#print(xtemp)
#output = net(xtemp, data)
#print('output test', output)

#mycrit = torch.nn.MSELoss();
mycrit = torch.nn.SmoothL1Loss();
#myoptimizer_nd = torch.optim.RMSprop(net_nd.parameters());
myoptimizer_nd = torch.optim.Adagrad(net_nd.parameters());
#myoptimizer = torch.optim.Adam(net.parameters(), lr=0.01, weight_decay=5e-4)

net_nd.train()

for epoch in range(nepoch):
  loss = 0;
  for batch_idx, (xtemp, ytemp) in enumerate(train_loader_nd):
   
    #forward step
    #print('one', xtemp)
    xtemp = torch.squeeze(xtemp)
    #print('two', xtemp)
    scores = net_nd(xtemp, data_nd)
    #remove extra tensor layers
    scores = torch.squeeze(scores)
    ytemp = torch.squeeze(ytemp)
    loss = mycrit(scores, ytemp) 

    #back step
    myoptimizer_nd.zero_grad();
    loss.backward();
    myoptimizer_nd.step();

  check_acc(test_loader_nd,net_nd, data_nd)
  print('epoch:', epoch, 'loss:', loss)

#Test ND Model

In [ ]:
%%time
#save
torch.save(net_nd.state_dict(), 'net_nd_gru.model')
#net_nd = GModel(data_nd)
#net_nd.load_state_dict(torch.load('net_nd_smooth_ada_200_512.model'))
#check
check_acc(train_loader_nd, net_nd, data_nd)
check_acc(test_loader_nd, net_nd, data_nd)

#Visual of Weights Helper

In [ ]:
import seaborn as sns

def visual_linear_weights(weights):
  print(weights)
  min = torch.min(weights)
  max = torch.max(weights)
  print('min', min, 'max', max)
  bound = torch.maximum(torch.abs(min), torch.abs(max))
  print('bound', bound)
  vmin = -bound
  vmax = bound
  epsilon = .01 if 0.01 <= vmax and .01 >= vmin else vmax/10
  low0 = (-epsilon - vmin)/(vmax - vmin)
  high0 = (epsilon + vmax)/(vmax - vmin)
  print('low', low0, 'high', high0)

  nodes = [0, low0, low0, high0, high0, 1]
  colors = ['green', 'blue', 'black', 'black', 'red', 'white']
  cmap = mpl.colors.LinearSegmentedColormap.from_list("", list(zip(nodes, colors)))

def heat_map(weights):
  weights = weights.detach().numpy()
  fig = plt.figure(figsize=(15,1));
  #plt.imshow(weights, cmap='hot')
  ax = sns.heatmap(weights, center=0)
  plt.show()

def heat_map2D(weights):
  weights = weights.detach().numpy()
  fig = plt.figure(figsize=(15,15));
  #plt.imshow(weights, cmap='hot')
  ax = sns.heatmap(weights, center=0)
  plt.show()  



In [ ]:
def count_nnz(weights):
    weights = weights.detach().numpy()
    m,n = weights.shape
    #print(m,n)
    counter = 0
    eps = .01
    for i in range(m):
        for j in range(n):
            val = weights[i][j]
            if(abs(val)<eps):
                counter += 1
                
    print('nnz', counter)
    print('percentage', (counter/(m*n)*100.0))
    
    

#Visual of Weights

In [ ]:
#nat order
print('NAT ORDER')
heat_map(net_nat.layer3.weight.cpu())
heat_map(net_nat.layer4.weight.cpu())
heat_map2D(net_nat.layer1.weight.squeeze().t().cpu())
heat_map2D(net_nat.layer2.weight.squeeze().t().cpu())
#rcm order
print('RCM ORDER')
heat_map(net_rcm.layer3.weight.cpu())
heat_map(net_rcm.layer4.weight.cpu())
heat_map2D(net_rcm.layer1.weight.squeeze().t().cpu())
heat_map2D(net_rcm.layer2.weight.squeeze().t().cpu())
#amd order
print('AMD ORDER')
heat_map(net_amd.layer3.weight.cpu())
heat_map(net_amd.layer4.weight.cpu())
heat_map2D(net_amd.layer1.weight.squeeze().t().cpu())
heat_map2D(net_amd.layer2.weight.squeeze().t().cpu())
#nd order
print('ND ORDER')
heat_map(net_nd.layer3.weight.cpu())
heat_map(net_nd.layer4.weight.cpu())
heat_map2D(net_nd.layer1.weight.squeeze().t().cpu())
heat_map2D(net_nd.layer2.weight.squeeze().t().cpu())




#Prediction Model Validate

In [ ]:
def PredictNNZ(loader_array):
  #loaders hold the information
  #data hold the parameter meta
  eps = 10;

  #Nat Loader
  loader = loader_array[0]
  loader_data = loader.dataset

  #put models into eval mode
  net_nat.eval()
  net_rcm.eval()
  net_amd.eval()
  net_nd.eval()
  
  #variables
  ntotal = 0
  ncorrect = 0
  nnotcorrect = 0
  avg_diff = 0
  percent_avg_diff = 0
  with torch.no_grad():
    for index in range(len(loader_data)):
      #print('index', index)
      ntotal += 1
      [x,y] = loader_data[index]
      #x = torch.tensor(x)


      acc_score = []
      acc_score_min_l = []
      acc_order = []
      #print('start value', acc_min_score)
      for l_index in range(len(loader_array)):
        temp_loader = loader_array[l_index]
        [tempx_loader, tempy_loader] = temp_loader.dataset[index]
        temp_v = tempy_loader
        acc_score.append(temp_v)
      acc_score_min = min(acc_score)
      eps = acc_score_min*.1
      for l_index in range(len(loader_array)):
        if((acc_score[l_index] <= acc_score_min +eps) and (acc_score[l_index] >= acc_score_min - eps)):
          acc_order.append(l_index)
      #print(acc_order)
      #print('Min: ', acc_score_min - eps, ' ', acc_score_min, ' ', acc_score_min + eps)
      #print('Order: ', acc_score)

        #print('Order: ', l_index, ' NNZ ', temp_v)
        #temp_v = loader_array[l_index][index][1]
        #temp_v = torch.squeeze(temp_v)
        #if(temp_v < acc_min_score):
          #acc_min_score = temp_v
          #acc_order = l_index
          #print('Index: ', index, ' Min: ', acc_min_score, ' Order: ', l_index)


      scores_nat = net_nat(x, data_nat).squeeze().item()
      scores_rcm = net_rcm(x, data_rcm).squeeze().item()
      scores_amd = net_amd(x, data_rcm).squeeze().item()
      scores_nd  = net_nd(x, data_nd).squeeze().item()
      #print('Predicted NNZ NAT: ', scores_nat, ' RCM: ', scores_rcm, ' AMD: ', scores_amd, ' ND: ', scores_nd)
      pred_min_score = scores_nat
      pred_order = 0
      if(scores_rcm < pred_min_score):
        pred_min_score = scores_rcm
        pred_order = 1
      if(scores_amd < pred_min_score):
        pred_min_score = scores_amd
        pred_order = 2
      if(scores_nd < pred_min_score):
        pred_min_score = scores_nd
        pred_order = 3

        
      nnz_diff = abs(pred_min_score - acc_score_min)
      avg_diff += nnz_diff
      percent_avg_diff += ((nnz_diff/acc_score_min)*100)
      #print('Index: ', index, ' Acc NNZ: ', acc_min_score, ' Acc Order: ', acc_order, ' Pred NNZ: ', pred_min_score, ' Pred Order: ', pred_order )
      if(pred_order in acc_order):
        ncorrect +=1
        #print('Correct')
      else:
        nnotcorrect +=1
        #print('Not  Corect')
      #print('Index: ', index, ' Acc Order: ', acc_order, ' Pred NNZ: ', pred_min_score, ' Pred Order: ', pred_order )

 

  #print('Total: ', ntotal, ' Correct: ', ncorrect, ' Not Correct: ', nnotcorrect, ' Percent Correct ', ncorrect/ntotal)
  #print('Avg Diff: ', avg_diff/ntotal, ' Percent Avg Diff: ', percent_avg_diff/ntotal)  
    
  return ntotal, ncorrect, nnotcorrect

In [ ]:
%%time
train_loader_nat.shuffle = False; train_loader_rcm.shuffle = False; train_loader_amd.shuffle = False; train_loader_nd.shuffle = False;
loader_array_train = [train_loader_nat, train_loader_rcm, train_loader_amd, train_loader_nd]
%time PredictNNZ(loader_array_train)
test_loader_nat.shuffle = False; test_loader_rcm.shuffle = False; test_loader_amd.shuffle = False; test_loader_nd.shuffle = False;
loader_array_test = [test_loader_nat, test_loader_rcm, test_loader_amd, test_loader_nd]
%time PredictNNZ(loader_array_test)

In [ ]:
def PredictNNZ_More(loader_array):
  #loaders hold the information
  #data hold the parameter meta
  eps = 10;

  #Nat Loader
  loader = loader_array[0]
  loader_data = loader.dataset

  #put models into eval mode
  net_nat.eval()
  net_rcm.eval()
  net_amd.eval()
  net_nd.eval()
  
  #totals --- more for evaluation table
  nat_obs = [0]*4;
  rcm_obs = [0]*4; 
  amd_obs = [0]*4;
  nd_obs  = [0]*4;

  obs_dist = [0]*4;


    
  #variables
  ntotal = 0
  ncorrect = 0
  nnotcorrect = 0
  avg_diff = 0
  percent_avg_diff = 0
  with torch.no_grad():
    for index in range(len(loader_data)):
      #print('index', index)
      ntotal += 1
      [x,y] = loader_data[index]
      #x = torch.tensor(x)


      acc_score = []
      acc_score_min_l = []
      acc_order = []
      #print('start value', acc_min_score)
      for l_index in range(len(loader_array)):
        temp_loader = loader_array[l_index]
        [tempx_loader, tempy_loader] = temp_loader.dataset[index]
        temp_v = tempy_loader
        acc_score.append(temp_v)
      acc_score_min = min(acc_score)
    
      obs_local_min = [];
      for l_index in range(len(loader_array)):
            if (acc_score[l_index] == acc_score_min):
                obs_local_min.append(l_index);
                obs_dist[l_index] = obs_dist[l_index]+1;
                
    
    
      eps = acc_score_min*.20
      for l_index in range(len(loader_array)):
        if((acc_score[l_index] <= acc_score_min +eps) and (acc_score[l_index] >= acc_score_min - eps)):
          acc_order.append(l_index)
      #print(acc_order)
      #print('Min: ', acc_score_min - eps, ' ', acc_score_min, ' ', acc_score_min + eps)
      #print('Order: ', acc_score)

        #print('Order: ', l_index, ' NNZ ', temp_v)
        #temp_v = loader_array[l_index][index][1]
        #temp_v = torch.squeeze(temp_v)
        #if(temp_v < acc_min_score):
          #acc_min_score = temp_v
          #acc_order = l_index
          #print('Index: ', index, ' Min: ', acc_min_score, ' Order: ', l_index)


      scores_nat = net_nat(x, data_nat).squeeze().item()
      scores_rcm = net_rcm(x, data_rcm).squeeze().item()
      scores_amd = net_amd(x, data_rcm).squeeze().item()
      scores_nd  = net_nd(x, data_nd).squeeze().item()
      #print('Predicted NNZ NAT: ', scores_nat, ' RCM: ', scores_rcm, ' AMD: ', scores_amd, ' ND: ', scores_nd)
      pred_min_score = scores_nat
      pred_order = 0
      if(scores_rcm < pred_min_score):
        pred_min_score = scores_rcm
        pred_order = 1
      if(scores_amd < pred_min_score):
        pred_min_score = scores_amd
        pred_order = 2
      if(scores_nd < pred_min_score):
        pred_min_score = scores_nd
        pred_order = 3

      #add more stuff
      #if(0 in obs_local_min): #NAT observed
      #      nat_obs[pred_order] = nat_obs[pred_order] + 1;
      #if(1 in obs_local_min): #RCM observed
      #      rcm_obs[pred_order] = rcm_obs[pred_order] + 1;
      #if(2 in obs_local_min): #AMD observed
      #      amd_obs[pred_order] = amd_obs[pred_order] + 1;
      #if(3 in obs_local_min): #ND observed
      #      nd_obs[pred_order] = nd_obs[pred_order] + 1;
            
            
      if(pred_order in obs_local_min):
        if(0 in obs_local_min):
            nat_obs[0] = nat_obs[0]+1;
        if(1 in obs_local_min):
            rcm_obs[1] = rcm_obs[1]+1;
        if (2 in obs_local_min):
            amd_obs[2] = amd_obs[2]+1;
        if (3 in obs_local_min):
            nd_obs[3] = nd_obs[3] + 1;
      else:
        if(0 in obs_local_min):
            nat_obs[pred_order] = nat_obs[pred_order]+1;
        if(1 in obs_local_min):
            rcm_obs[pred_order] = rcm_obs[pred_order]+1;
        if (2 in obs_local_min):
            amd_obs[pred_order] = amd_obs[pred_order]+1;
        if (3 in obs_local_min):
            nd_obs[pred_order] = nd_obs[pred_order] + 1;
            
        
        
      nnz_diff = abs(pred_min_score - acc_score_min)
      avg_diff += nnz_diff
      percent_avg_diff += ((nnz_diff/acc_score_min)*100)
      #print('Index: ', index, ' Acc NNZ: ', acc_min_score, ' Acc Order: ', acc_order, ' Pred NNZ: ', pred_min_score, ' Pred Order: ', pred_order )
      if(pred_order in acc_order):
        ncorrect +=1
        #print('Correct')
      else:
        nnotcorrect +=1
        #print('Not  Corect')
      #print('Index: ', index, ' Acc Order: ', acc_order, ' Pred NNZ: ', pred_min_score, ' Pred Order: ', pred_order )


  print('Eval Matrix')
  print('NAT:', end='')
  for kk in range(4):
    print(nat_obs[kk], ' , ', end='')
  print("")
  print("RCM:", end='')
  for kk in range(4):
    print(rcm_obs[kk], ', ', end='')
  print("")
  print("AMD:", end='')
  for kk in range(4):
    print(amd_obs[kk], ', ', end='')
  print("")
  print("ND: ", end='')
  for kk in range(4):
    print(nd_obs[kk], ', ', end='')
  print("")


 
  print('NAT: ', nat_obs[0], rcm_obs[0], amd_obs[0], nd_obs[0], (nat_obs[0])/(nat_obs[0]+rcm_obs[0]+amd_obs[0]+nd_obs[0]))
  print('RCM: ', nat_obs[1], rcm_obs[1], amd_obs[1], nd_obs[1], (rcm_obs[1])/(nat_obs[1]+rcm_obs[1]+amd_obs[1]+nd_obs[1]))
  print('AMD: ', nat_obs[2], rcm_obs[2], amd_obs[2], nd_obs[2], (amd_obs[2])/(nat_obs[2]+rcm_obs[2]+amd_obs[2]+nd_obs[2]))
  print('ND: ', nat_obs[3], rcm_obs[3], amd_obs[3], nd_obs[3], (nd_obs[3])/(nat_obs[3]+rcm_obs[3]+amd_obs[3]+nd_obs[3]))
  print('     ', nat_obs[0]/sum(nat_obs), rcm_obs[1]/sum(rcm_obs), amd_obs[2]/sum(amd_obs), nd_obs[3]/sum(nd_obs))


  print('Distribution')
  print(obs_dist)
    

  print('Total: ', ntotal, ' Correct: ', ncorrect, ' Not Correct: ', nnotcorrect, ' Percent Correct ', ncorrect/ntotal)
  print('Avg Diff: ', avg_diff/ntotal, ' Percent Avg Diff: ', percent_avg_diff/ntotal)  

In [ ]:
loader_array_train = [train_loader_nat, train_loader_rcm, train_loader_amd, train_loader_nd]
PredictNNZ_More(loader_array_train)

loader_array_test = [test_loader_nat, test_loader_rcm, test_loader_amd, test_loader_nd]
PredictNNZ_More(loader_array_test)